In [1]:
import random
import matplotlib.pyplot as plt 
from matplotlib.pyplot import cm
import numpy as np
import pandas as pd
from pandas import DataFrame
from scipy.stats import multivariate_normal
from sklearn.neighbors import KernelDensity
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn import metrics
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.preprocessing import StandardScaler
from mlxtend.plotting import plot_decision_regions
import matplotlib.pyplot as plt
from sklearn import datasets
from sklearn.neighbors import KNeighborsClassifier
import sklearn.datasets as skdata
from scipy.spatial import distance
import statistics

# Teoria da decisão Bayesiana

+ $P(C_1|x)$ - Probabilidade de atributos x pertencer à classe $C_1$.
+ $P(C_2|x)$ - Probabilidade de atributos x pertencer à classe $C_2$.

Se $P(C_1|x) > P(C_2|x)$ então classifique $x \in C_1$, o mesmo vale
para o caso inverso.

Pelo teorema de Bayes temos 
$$ P(C_j| x) = \frac{P(x|C_j)P(C_j)}{p(x)}, j \in \mathbb{N}^*$$
onde $P(x|C_j)$ é uma função de densidade de probabilidade e $P(C_j)$ é uma distribuição de variaveis aleatórias discretas.
Assumimos $P(C_j|) \sim p(x | C_j)$


## Probabilidade de erro na classificação

$$p_e = p(X \in R_1, C_2) + p(X \in R_2, C_1) = p(X \in R_1|C_2)P(C_2) + p(X \in R_2| C_1)P(C_1)$$
$$p_e = \int_{R_1} p(X|C_2)P(C_2)dx + \int_{R_2}p(X|C_1)P(C_1)dx$$

mas $P(C_j|x) = \frac{p(X|C_j)P(C_j)}{p(X)} \longrightarrow P(C_j|X)p(X)$

$$p_e = \int_{R_1} p(C_2|X)p(X)dx + \int_{R_2} P(C_1|X)p(X) dx$$

$$ P(C_1) = \int_{-\infty}^{\infty} P(x_i c_i) dx = \int_{-\infty}^{\infty}P(C_1|X)p(X)dx = \int_{R_1}P(C_1|X)p(X)dx + \int_{R_2}P(C_1|X)p(X)dx$$

Substituindo em $p_e$: 

$$ p_e = \int_{R_1} P(C_2|X)p(X) dx + P(C_1) - \int_{R_1}P(C_1|X)p(X)dx$$

$$ p_e = P(C_1) - \int_{R_1} \left[ P(C_1|X) - P(C_2|X)\right] p(X) dx $$




O erro será mínimo se: $ P(C_1|X) - P(C_2|X) > 0 $ em $R_1$. 

## Classificação Bayesiana: estimação paramétrica


Sejam $X_1, X_2, \cdots, X_n$ uma amostra aleatória da população, assumindo independência dos elementos da amostra 
$$p(X; \theta) = p(x_1, x_2, \cdots,x_n) = \prod_{i = 1}^{n} p(x_i; \theta) $$
o estimador de máxima verossimilhança é dado por 
$$\hat{\theta} = \text{argmax}_{\theta}\prod_{i = 1}^{n} p(x_i; \theta)$$
ou
$$ L(\theta) = \log p(X; \theta) = \sum_{i = 1}^{n} \log p(X; \theta) $$


Estimadores de verossimilhanã são assintoticamente não-viesados, ou seja, a média, para um número de amostras muito grande converge para um número real: $\lim_{n \to \infty} E[\hat{\theta}] = \theta_0$.
  

## Classificação não-paramétrica
Considerando para uma função de 1 variável, temos que 
$$ \hat{f}_{h} (x) = \frac{1}{n} \sum_{i = 1}^{n} K_h (x - x_i) = \frac{1}{nh} \sum_{i = 1}^{n} K \left( \frac{x - x_i}{h} \right)$$
 onde $K$ é uma função kernel calculada em cada ponto e $h$ é chamado de hiperparametro.


# Implementação do classificador Bayesiano

In [2]:
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Introdução à Ciência de Dados/data/Vehicle.csv')
data = data.dropna(axis = 'rows')

classes = np.array(pd.unique(data[data.columns[-1]]),  dtype = str)

data = data.to_numpy()
nrow, ncol = data.shape

y = data[:, -1]
X = data[:, 0:ncol - 1]

In [3]:
scaler = StandardScaler().fit(X)
X = scaler.transform(X)

In [4]:
p = 0.6
x_train, x_test, y_train, y_test = train_test_split(X, y, train_size = p, random_state = 302)

## Caso paramétrico
Assumindo que cada variável está distribuida a partir de uma distribuição Normal.

In [5]:
P = pd.DataFrame(data=np.zeros((x_test.shape[0], len(classes))), columns = classes) 

Pc = np.zeros(len(classes))
for i in np.arange(0, len(classes)):
    elements = tuple(np.where(y_train == classes[i]))
    Pc[i] = len(elements)/len(y_train)
    Z = x_train[elements,:][0]
    m = np.mean(Z, axis = 0)
    cv = np.cov(np.transpose(Z))
    for j in np.arange(0,x_test.shape[0]):
        x = x_test[j,:]
        pj = multivariate_normal.pdf(x, mean=m, cov=cv, allow_singular=True)
        P[classes[i]][j] = pj*Pc[i]


In [6]:
y_pred = []
#np.array(test_x.shape[0], dtype=str)
for i in np.arange(0, x_test.shape[0]):
    c = np.argmax(np.array(P.iloc[[i]]))
    y_pred.append(classes[c])
y_pred = np.array(y_pred, dtype=str)
# print(y_pred)

In [7]:
score = accuracy_score(y_pred, y_test)
print('Accuracy:', score)

Accuracy: 0.8584070796460177


## Caso não paramétrico
Para o caso unidimensional seja uma amostra identicamente distribuida de acordo com uma função $f$ não conhecida. Usamos um estimador (Kernel density estimator):
$$ \hat{f}_{h} (x) = \frac{1}{n} \sum_{i = 1}^{n} K_h (x - x_i) = \frac{1}{nh} \sum_{i = 1}^{n} K \left( \frac{x - x_i}{h} \right)$$
nesse caso a estimação depende do parâmetro $h$, que controla a abertura da função.



In [8]:
p = 0.8 # fraction of elements in the test set
x_train, x_test, y_train, y_test = train_test_split(X, y, train_size = p, random_state = 42)

In [9]:
p = pd.DataFrame(data=np.zeros((x_train.shape[0], len(classes))), columns = classes)
pc = np.zeros(len(classes))
  
h = 0.6

for i in np.arange(0, len(classes)):
    elements = tuple(np.where(y_train == classes[i]))
    pc[i] = len(elements) / len(y_train)
    z = x_train[elements, :][0]
    kde = KernelDensity(kernel = 'gaussian', bandwidth=h).fit(z)
    for j in np.arange(0, x_test.shape[0]):
      x = x_test[j, :]
      x = x.reshape((1, len(x)))
      pj = np.exp(kde.score_samples(x))
      p[classes[i]][j] = pj * pc[i]
y_pred = []

for i in np.arange(0, x_test.shape[0]):
  c = np.argmax(np.array(p.iloc[[i]]))
  y_pred.append(classes[c])
  
y_pred = np.array(y_pred, dtype = str)

score = accuracy_score(y_pred, y_test)
print("Acuracia: ", score)

Acuracia:  0.7176470588235294


In [10]:
v_accuracy = []
v_h = np.linspace(0.1, 5, 100)

for h in v_h:
  p = pd.DataFrame(data=np.zeros((x_train.shape[0], len(classes))), columns = classes)

  pc = np.zeros(len(classes))

  for i in np.arange(0, len(classes)):
    elements = tuple(np.where(y_train == classes[i]))
    pc[i] = len(elements) / len(y_pred)

    z = x_train[elements, :][0]
    kde  = KernelDensity(kernel = 'gaussian', bandwidth = h).fit(z)

    for j in np.arange(0, x_test.shape[0]):
      x = x_test[j, :]
      x = x.reshape((1, len(x)))
      pj = np.exp(kde.score_samples(x))
      p[classes[i]][j] = pj * pc[i]
  
  y_pred = []  
  for i in np.arange(0, x_test.shape[0]):
    c = np.argmax(np.array(p.iloc[[i]]))
    y_pred.append(classes[c])

  y_pred = np.array(y_pred, dtype = int)
  score = accuracy_score(y_pred, y_test)
  v_accuracy.append(score)


ValueError: ignored

In [ ]:
plt.figure(figsize=(10,4))
plt.plot(v_h, v_accuracy, 'o--', color = 'red', linewidth=1)
plt.xlabel('h', fontsize = 15)
plt.ylabel('Acurácia)', fontsize = 15)
plt.show()

In [ ]:
best_h = v_h[np.argmax(v_accuracy)]
print("Melhor h: ", best_h)

# Classificador Naive Bayes

Não conhecemos a distribuição de probabilidade conjunta 
$$ P(C_i| x) = \frac{p(X|C_i) P(C_i)}{p(X)}  $$
então fazemos inferência assumindi primeiramente a independência entre os 
atributos, ou seja 
$$P(C_i|x) = \frac{p(x|C_i)P(C_i)}{p(x)}$$ 
$$ p(x|C_i) = \prod_{j = 1}^{d} p(x_j|C_i), i = 1, 2, \cdots, k $$

e a classificação usa da regra de Bayes 
$$ C_m = \underbrace{\text{argmax}}_{i \in \{1, \cdots, M\}} p(C_i) \prod_{j = 1}^{d} p(x_j | C_i) $$



## Implementando o classificador Naive Bayes assumindo distribuição normal
Vamos assumir que a distribuição é do tipo normal, ou seja 
$$ p(x_j | C_i) = \frac{1}{\sqrt{2\pi \sigma_{C_i}}} exp \left[ - \frac{1}{2} \left( \frac{x_j - \mu_{C_i}}{\sigma_{C_i}} \right)^2 \right]$$

Assumindo a independência, o calculo de observação $q$ será 
$$ p(x_q | C_i) = \prod_{i = 1}^{d} p (x_{q, j} | C_i) , i = 1, 2, \cdots, k$$

a classificação de acordo com a classe mais provável será
 $$ \hat{y} = \text{arg} \underbrace{\text{max}}_{i = 1, \cdots, k} P(C_i) \prod_{j = 1}^{d} p(x_{q, j} | C_i)$$


# Implementação do classificador Naive Bayes

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Introdução à Ciência de Dados/data/Vehicle.csv')
data = data.dropna(axis = 'rows')

classes = np.array(pd.unique(data[data.columns[-1]]),  dtype = str)

In [ ]:
data = data.to_numpy()
nrow, ncol = data.shape

y = data[:, -1]
X = data[:, 0:ncol - 1]

In [ ]:
p = 0.6
x_train, x_test, y_train, y_test = train_test_split(X, y, train_size = p, random_state = 302)

## Definindo uma função de verossimilhança

In [ ]:
def likelyhood(y, z):
  def gaussian(x, mu, sig):
    return np.exp(-np.power(x - mu, 2.) / (2 * np.power(sig, 2.)))
  prob = 1
  for j in np.arange(0, z.shape[1]):
    m = np.mean(z[:, j])
    s = np.std(z[:, j])
    prob = prob * gaussian(y[j], m, s)
  return prob

In [ ]:
p = pd.DataFrame(data = np.zeros((x_test.shape[0], len(classes))), columns = classes)
for i in np.arange(0, len(classes)):
  elements = tuple(np.where(y_train == classes[i]))
  z = x_train[elements, :][0]
  for j in np.arange(0, x_test.shape[0]):
    xj = x_test[j, :]
    pj = likelyhood(xj, z)
    p[classes[i]][j] = pj * len(elements) / x_train.shape[0]

In [ ]:
p.head(10)

In [ ]:
y_pred = []
for i in np.arange(0, p.shape[0]):
    c = np.argmax(np.array(p.iloc[[i]]))
    y_pred.append(p.columns[c])
y_pred = np.array(y_pred, dtype=str)

score = accuracy_score(y_pred, y_test)
print('Accuracy:', score)

## Usando os métodos já implementados pelo scikit-learn

In [ ]:
model = GaussianNB()
model.fit(x_train, y_train)

y_pred = model.predict(x_test)
score = accuracy_score(y_pred, y_test)
print('Accuracy:', score)

In [ ]:
model = BernoulliNB()

model.fit(x_train, y_train)

y_pred = model.predict(x_test)
score = accuracy_score(y_pred, y_test)
print('Accuracy:', score)

In [ ]:
# Gera os dados em duas dimensões
n_samples = 100 # número de observações
# centro dos grupos
centers = [(-4, 0), (0, 0), (3, 3)]
X, y = skdata.make_blobs(n_samples=100, n_features=2, cluster_std=1.0, centers=centers, 
                         shuffle=False, random_state=42)

# monta a matrix de atributos
d = np.column_stack((X,np.transpose(y)))
# converte para o formato dataframe do Pandas
data = DataFrame(data = d, columns=['X1', 'X2', 'y'])
features_names = ['X1', 'X2']
class_labels = np.unique(y)

In [ ]:
colors = ['red', 'blue', 'green', 'black']
aux = 0
for c in class_labels:
    ind = np.where(y == c)
    plt.scatter(X[ind,0][0], X[ind,1][0], color = colors[aux], label = c)
    aux = aux + 1
plt.legend()
plt.show()

# Training a classifier
model = GaussianNB()
model.fit(X, y)

# Plotting decision regions
plot_decision_regions(X, y, clf=model, legend=2)

plt.xlabel('X1')
plt.ylabel('X2')
plt.title('Decision Regions')
plt.show()

# KNN
É criado um vetor de atributos para cada classe e então calculado as distâncias entre os atributos.

## Definição da distância

É comparada várias medidas de distância (euclidiana, manthattan ...)

### Medidas de proximidade
- **Medida de similaridade**: $d(X_i, X_i)$ é máxima.
  + $s(p, q) = 1, p = q$;
  + $s(p, q) = s(q, p)$
- **Medida de dissimilaridade**: $d(X_i, X_i) = 0$.
  + $d(p, q) \ge 0, ∀ p, q$ e $d(p, q) = 0$, se e só se $p = q$.
  + $d(p, q) = d(q, p)$.
  + $d(p, r) \le d(p, q) + d(q, r), ∀ p, q, r$ onde $d(p,q)$ é a distância de dissimilaridade entre $p$ e $q$.

## Métricas de distância 
### **Dissimilaridade**
- Euclidiana 
$$ D(X, Y) = \sqrt{ \sum_{i = 1}^{n} (x_i - y_i)^2}, x, y \in [0, \infty)$$

- Minkowski 
$$ D(X, Y) = \left( \sum_{i = 1}^{n} | x_i - y_i |^p\right)^{\frac{1}{p}}, x, y \in [0, \infty) $$

### **Similaridade**
- Cosseno 
$$ D(X, Y) = \frac{\sum_{i = 1}^{n} x_i y_i}{\sqrt{\sum_{i = 1}^{n} (x_i)^2}\sqrt{\sum_{i = 1}^{n} (y_i)^2}} x, y \in [0, 1]$$ 

- Pearson 
$$ D(X, Y) = \frac{\sum_{i = 1}^{n} (x_i - \bar{x})(y_i - \bar{y})}{\sqrt{\sum_{i = 1}^{n} (x_i - \bar{x})^2} \sqrt{\sum_{i = 1}^{n} (y_i - \bar{y})^2}}, x, y \in [-1, 1] $$

## Algoritmo KNN

+ Identifique os K vizinhos mais proximos do vetor de atributos $X$ que se quer classificar.
+ Determine o número de vizinhos em cada classe.
+ Classifique $X$ como pertencente à classe que resultou em maior número de vizinhos

$$ p (y = j | X ) = \frac{1}{k} \sum_{i \in R}\{y_i = j\} $$

## Implementação do KNN

In [ ]:
def knn(x_train, y_train, x_test, k):
    distances = [] 
    x1 = x_test 
    for x2 in x_train: 
        dist = distance.euclidean(x1,x2)
        distances.append(dist)
    indices = []
    cl = []
    for i in range(0,k):
        ind = np.argmin(distances)
        #print('distance:', distances[ind],'index:', ind, 'class:', y_train[ind])
        distances[ind] = np.max(distances) 
        indices.append(ind)
        cl.append(y_train[ind])
    print("Classes:",cl)
    classification = statistics.mode(cl)
    return classification


In [ ]:
k=3 # numero de vizinhos
x_train = np.array([[1,0.5],[0.8,0.8],[1.2,1.4],[0.6,0.4],[0.4,1.2],[1.5,1]])
y_train = np.array(['white','gray','white','gray','gray','white'], dtype = 'str')
x_test = np.array([1,1])
# realiza a classificacao
cl = knn(x_train, y_train, x_test, k)
print("Classification:", cl)

In [ ]:
from scipy.spatial import Voronoi, voronoi_plot_2d
from sklearn import datasets
from sklearn.neighbors import KNeighborsClassifier
import sklearn.datasets as skdata
from matplotlib import pyplot

plt.scatter(x_train[:,0],x_train[:,1],c=y_train, s=150, marker='o', edgecolor='black')
plt.plot(x_test[0],x_test[1], marker='s', markersize=15, color="black")
plt.xlim(0.2,1.6)
plt.ylim(0,1.6)
plt.savefig('knn.eps')

plt.show()

In [ ]:
from scipy.spatial import Voronoi, voronoi_plot_2d
from sklearn import datasets
from sklearn.neighbors import KNeighborsClassifier
import sklearn.datasets as skdata
from matplotlib import pyplot

In [ ]:
# Gera os dados em duas dimensões
n_samples = 100 # número de observações
# centro dos grupos
centers = [(0, 0), (-2, -2), (2,0)]
X, y = skdata.make_blobs(n_samples=100, n_features=2, cluster_std=1.0, centers=centers, 
                         shuffle=False, random_state=42)

# monta a matrix de atributos
d = np.column_stack((X,np.transpose(y)))
# converte para o formato dataframe do Pandas
data = DataFrame(data = d, columns=['X1', 'X2', 'y'])
features_names = ['X1', 'X2']
class_labels = np.unique(y)

# mostra os dados e colori de acordo com as classes
colors = ['red', 'blue', 'green', 'black']
aux = 0
for c in class_labels:
    ind = np.where(y == c)
    plt.scatter(X[ind,0][0], X[ind,1][0], color = colors[aux], label = c)
    aux = aux + 1
plt.savefig('knn_ex.eps')
plt.legend()
plt.show()

In [ ]:
# mostra as regiões de separação para diversos valores de k
vk = [1,5,10,20,int(n_samples/2)]
for k in vk:
    # Training a classifier
    model = KNeighborsClassifier(n_neighbors=k, metric = 'euclidean')
    model.fit(X, y)
    # Plotting decision regions
    plot_decision_regions(X, y, clf=model, legend=2)
    plt.xlabel('X1')
    plt.ylabel('X2')
    plt.title('Decision Regions: k = '+str(k))
    #plt.savefig('knn_' + str(k)+'.eps')
    plt.show()

## Escolha do melhor $k$

In [ ]:
from sklearn.model_selection import cross_validate

# Gera os dados em duas dimensões
n_samples = 100 # número de observações
# centro dos grupos
centers = [(0, 0), (-2, -2), (2,0)]
X, y = skdata.make_blobs(n_samples=100, n_features=2, cluster_std=1.0, centers=centers, 
                         shuffle=False, random_state=42)

# monta a matrix de atributos
d = np.column_stack((X,np.transpose(y)))
# converte para o formato dataframe do Pandas
data = DataFrame(data = d, columns=['X1', 'X2', 'y'])
features_names = ['X1', 'X2']
class_labels = np.unique(y)

nkf = 5 #number of folds
vk = [] # armazena os valores de k
vscore = []
for k in range(1, 20):
    model = KNeighborsClassifier(n_neighbors=k, metric = 'euclidean')
    # realiza a validação cruzada
    cv = cross_validate(model, X, y, cv=nkf)
    #print('k:', k, 'accurace:', cv['test_score'].mean())
    vscore.append(cv['test_score'].mean()) 
    vk.append(k)

plt.figure(figsize=(6,4))
plt.plot(vk, vscore, '-bo')
plt.xlabel('k', fontsize = 15)
plt.ylabel('Acuracy', fontsize = 15)
plt.show(True)
best_k = np.argmax(vscore)+1
print('Melhor k:', best_k)